In [2]:
# Guarda la información de cada municipio en un DataFrame.

import os

import pandas as pd
import numpy as np

import geopandas as gpd

In [28]:
RCP = ["RCP4p5", "RCP8p5"]
dirs = ["Actual"] + RCP
path = "../../../resultados/RandomTree/data/"

path_r = [path + x for x in dirs ]
path_shp = "../../../datos/Mapas/Municipios/mun20gw.shp"

fut = [2030, 2050, 2070]

# Si no existen las carpetas, las crea.
if not os.path.exists(path):
    os.mkdir(path)
for f in path_r:
    if not os.path.exists(f):
        os.mkdir(f)

# Cargamos los municipios.
gdf = gpd.read_file( path_shp )

# Obtenemos el contorno de los municipios.
gdf["boundary"] = gdf.boundary

# Obtenemos el centroide de los municipios
# y separamos su latitud y longitud.
gdf["centroid"] = ( gdf.to_crs("epsg:6372")
    .centroid.to_crs("epsg:4326") )
gdf["lon"] = gdf["centroid"].x
gdf["lat"] = gdf["centroid"].y

In [33]:
# DataFrame para los datos actuales.

# Columnas a retirar del GeoDataFrame.
drop = ["CVEGEO", "PERIMETER", "COV_",
    "COV_ID", "geometry", "boundary", "centroid"]

# Columnas de variables explicativas.
cols = ["Consumo_1", "Usuarios_1", "Consumo_DAC",
    "Usuarios_DAC", "T_max", "T_min", "T_mean",
    "HDD_mean", "CDD_mean", "HDD_p10", "CDD_p90",
    "Pre", "Pre_Tmean", "Densidad_población",
    "PCI", "$luz", "$GLP", "Población", "PIB"]

# Convertimos de GeoDataFrame a DataFrame. 
df_0 = pd.DataFrame(gdf.drop(drop, axis = 1))
df_0["Año"] = 2010
df_0[cols] = np.nan

# Agregamos una fila para cada año en cada municipio.
df = df_0.copy()
for i in range(2011, 2017):
    df_0["Año"] = i
    df = df.append( df_0, ignore_index = True )
df = df[ list(df.columns[0:4]) + cols[0:4]
    + list(df.columns[5:7]) + cols[4:-2]
    + cols[-2:] + list(df.columns[7:8])
    + list(df.columns[4:5]) ]
df.to_csv(path_r[0] + "/data_0.csv",
    index = False, encoding = 'utf8')

In [32]:
# DataFrame para los datos futuros.

# Columnas a retirar del GeoDataFrame.
drop = ["CVEGEO", "PERIMETER", "COV_",
    "COV_ID", "geometry", "boundary", "centroid"]

# Columnas de variables explicativas.
cols = ["Consumo_1", "Usuarios_1", "Consumo_DAC",
    "Usuarios_DAC", "T_max", "T_min", "T_mean",
    "HDD_mean", "CDD_mean", "HDD_p10", "CDD_p90",
    "Pre", "Pre_Tmean", "Densidad_población",
    "PCI", "$luz", "$GLP", "Población", "PIB"]

# Convertimos de GeoDataFrame a DataFrame. 
df = pd.DataFrame(gdf.drop(drop, axis = 1))
df[cols] = np.nan

# Agregamos una fila para cada año en cada municipio.
df = df[ list(df.columns[0:4]) + cols[0:4]
    + list(df.columns[5:7]) + cols[4:-6]
    + cols[-2:] + cols[-6:-2]
    + list(df.columns[4:5]) ]
for f in fut:
    df["Año"] = f
    for r in path_r[1:]:
        df.to_csv(r + "/data_0." + str(f) + ".csv",
            index = False, encoding = 'utf8')